In [129]:
import os
import fastf1 as ff1
import fastf1.plotting
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Create cache directory if it doesn't exist
cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

ff1.Cache.enable_cache(cache_dir)

In [124]:
schedule = ff1.get_event_schedule(2025)
schedule[schedule['EventFormat'] == 'Conventional']

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport


In [126]:
def get_event_data(year, event_name):
    '''
    Get each driver fastest lap for a conventional event (FP1, FP2, FP3, Q, R)

    returns:
    dataframe of all drivers fastest laps for the event
    '''
    session_types = ['FP1', 'FP2', 'FP3', 'Q', 'R']
    final_results = {}

    for session_type in session_types:
        try:
            session = ff1.get_session(year, event_name, session_type)
            session.load()
        
            drivers = session.laps['Driver'].unique()
            for driver in drivers:
                if driver not in final_results:
                    final_results[driver] = {
                        'driver_number': session.laps.loc[session.laps['Driver'] == driver, 'DriverNumber'].iloc[0],
                        'year': year,
                        'event': event_name,
                        'team': session.laps.loc[session.laps['Driver'] == driver, 'Team'].iloc[0]
                    }
                driver_laps = session.laps.pick_driver(driver)
                    
                if not driver_laps.empty:
                    sector_times = driver_laps[['Sector1Time', 'Sector2Time', 'Sector3Time']].mean().dropna()             
                    total_times = sector_times.sum().total_seconds() if not sector_times.isna().any() else None

                    final_results[driver] = {
                        **final_results[driver],
                        f'{session_type}_sector1_time': sector_times['Sector1Time'].total_seconds() if 'Sector1Time' in sector_times else None,
                        f'{session_type}_sector2_time': sector_times['Sector2Time'].total_seconds() if 'Sector2Time' in sector_times else None,
                        f'{session_type}_sector3_time': sector_times['Sector3Time'].total_seconds() if 'Sector3Time' in sector_times else None,
                        f'{session_type}_total_time': total_times
                    }
            
            if session_type == 'R':
                result_positions = session.results[['ClassifiedPosition', 'Abbreviation']]
                result_positions_df = pd.DataFrame(result_positions).set_index('Abbreviation')
                for driver in drivers:
                    if driver in result_positions_df.index:
                        final_results[driver]['race_position'] = result_positions_df.loc[driver, 'ClassifiedPosition']
   
        except Exception as e:
            print(f"Could not load session {session_type} for {event_name} {year}: {e}")
            continue

    df = pd.DataFrame(final_results).T

    return df

In [127]:
australian_gp_2025_data = get_event_data(2025, 'Australian Grand Prix')
australian_gp_2025_data

core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '7', '10', '12', '14', '16', '18', '22', '23', '27', '30', '31', '44', '55', '63', '81', '87']
c:\Users\khanh\anaconda3\envs\DDSC\Lib\site-packages\fastf1\core.py:3183: FutureWarning: pick_driver is dep

,driver_number,year,event,team,FP1_sector1_time,FP1_sector2_time,FP1_sector3_time,FP1_total_time,FP2_sector1_time,FP2_sector2_time,...,FP3_total_time,Q_sector1_time,Q_sector2_time,Q_sector3_time,Q_total_time,R_sector1_time,R_sector2_time,R_sector3_time,R_total_time,race_position
VER,1,2025,Australian Grand Prix,Red Bull Racing,36.635523,22.741714,44.779777,104.157015,35.356863,22.138545,...,109.207585,38.196941,20.523529,36.088,94.80847,36.825607,23.512403,47.040719,107.378729,2
NOR,4,2025,Australian Grand Prix,McLaren,38.422,21.734285,43.457222,103.613507,34.548466,22.138633,...,108.397392,39.25265,20.98675,37.834642,98.074042,37.191232,23.721087,46.479192,107.391512,1
BOR,5,2025,Australian Grand Prix,Kick Sauber,39.730545,24.033818,47.448833,111.213196,35.56031,22.132275,...,112.398633,41.563923,20.917692,41.165888,103.647504,36.537272,23.252911,47.801733,107.591917,R
HAD,6,2025,Australian Grand Prix,Racing Bulls,36.79944,20.72076,45.530863,103.051063,35.5303,21.4769,...,108.71865,43.23,19.058416,37.75975,100.048166,None,None,None,0.0,R
DOO,7,2025,Australian Grand Prix,Alpine,37.25675,22.59121,45.592166,105.440127,34.896433,21.525066,...,116.110055,41.586866,19.894,39.05509,100.535957,None,None,None,0.0,R
GAS,10,2025,Australian Grand Prix,Alpine,34.497333,22.101181,44.111,100.709515,35.812733,21.967666,...,113.662822,40.8803,19.71045,38.077266,98.668016,36.575125,23.519842,47.66921,107.764177,11
ANT,12,2025,Australian Grand Prix,Mercedes,36.205913,22.138958,44.056428,102.401299,34.3628,21.651741,...,103.367126,37.631888,20.711555,38.636285,96.97973,36.392589,23.503333,47.519421,107.415343,4
ALO,14,2025,Australian Grand Prix,Aston Martin,32.323739,19.935391,42.47295,94.73208,32.146333,20.468,...,102.128944,41.518461,19.653923,37.73175,98.904134,36.02787,21.895437,46.217875,104.141183,R
LEC,16,2025,Australian Grand Prix,Ferrari,37.7957,21.196,43.399823,102.391523,34.659937,21.081312,...,116.384202,38.93255,19.64155,39.249923,97.824023,36.446892,23.270561,47.955543,107.672998,8
STR,18,2025,Australian Grand Prix,Aston Martin,36.30645,22.178947,47.387529,105.872926,33.774571,20.6695,...,102.945878,38.534466,19.763533,37.5125,95.810499,36.424017,23.536508,47.605614,107.56614,6


In [128]:
australian_gp_2025_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, VER to BEA
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   driver_number     20 non-null     object
 1   year              20 non-null     object
 2   event             20 non-null     object
 3   team              20 non-null     object
 4   FP1_sector1_time  20 non-null     object
 5   FP1_sector2_time  20 non-null     object
 6   FP1_sector3_time  20 non-null     object
 7   FP1_total_time    20 non-null     object
 8   FP2_sector1_time  19 non-null     object
 9   FP2_sector2_time  19 non-null     object
 10  FP2_sector3_time  19 non-null     object
 11  FP2_total_time    19 non-null     object
 12  FP3_sector1_time  20 non-null     object
 13  FP3_sector2_time  20 non-null     object
 14  FP3_sector3_time  20 non-null     object
 15  FP3_total_time    20 non-null     object
 16  Q_sector1_time    19 non-null     object
 17  Q_sector2_time    20